# TP3 - Calibration de caméras

OpenCV est une bibliothèque Open Source de vision par ordinateur très utilisée dans les domaines académiques et même dans l’industrie. Elle permet en effet d’utiliser de nombreuses méthodes et algorithmes à la pointe dans les domaines tels que :
- le traitement d’images ;
- l’analyse vidéo ;
- la détection d’objets et/ou de points d’intérêts ;
- etc.

OpenCV est une bibliothèque découpée en plusieurs modules, dans la séance d’aujourd’hui, nous allons utiliser les modules suivants :
- core : le module de base d’OpenCV contenant les classes de base (cv::Mat, etc.) ;
- highgui : module contenant les classes des interfaces utilisateurs (fenêtre, gestion clavier et souris, etc.) ;
- imgproc : module de traitement d’image (par exemple conversion d’une image couleur vers niveaux de gris, etc.) ;
- calib3d : module de calibration 3D et stéréoscopique (détection d’un échiquier, etc.).

Tout au long du TP merci de vous référer à la documentation OpenCV [https://docs.opencv.org/4.x/](https://docs.opencv.org/4.x/)


## 1/ Prise en main - Lecture vidéo et fichiers images

Dans cette partie, nous allons vous familiariser avec l’utilisation d’OpenCV. Pour ce faire,
nous allons vous faire écrire une application qui va :
- afficher les images issues d’une caméra ou de différents fichiers en boucle jusqu’à ce
que l’utilisateur appuie sur la touche 'esc' ;
- convertir une image couleur en niveaux de gris ;
- offrir la possibilité à l’utilisateur de choisir entre l’affichage de l’image en couleur ou
l’image en niveaux de gris ;
- libérer proprement les ressources avant de quitter votre programme.


### 1.1/ Squelette de l'application

A partir des exemples du TP2, créez un fichier OpenCV-calibration-part1.cpp (et le fichier
Cmake correspondant).
Commencez par inclure les entêtes OpenCV que nous allons utiliser :
```
// OpenCV core module (matrices, etc.)
#include <opencv2/core/core.hpp>
// OpenCV highgui: user interface, windows, etc.
#include <opencv2/highgui/highgui.hpp>
// OpenCV image processing (converting to grayscale, etc.)
#include <opencv2/imgproc/imgproc.hpp>
// IO
#include <iostream>
using namespace std;
using namespace cv;
```
Vous aurez également besoin de définir une constante représentant le code ASCII de la touche
```
’ESC’ et espace du clavier, que nous voulons pouvoir reconnaitre pour quitter l’application :
// the ASCCI code for the escape and space key
#define ESC_KEY 27
#define SPC_KEY 32
```
Nous avons besoin de définir une variable qui va nous permettre de récupérer une touche
pressée par l’utilisateur :
```
// A key that we use to store the user keyboard input
char key;
```
Définissez maintenant la méthode principale de notre application :
```
// The main function
int main(int argc, char** argv) {
    // define variables
    char key;

    // Starting the code

    // exiting the program
    return EXIT_SUCCESS;
}
```

In [7]:
import cv2
import numpy as np
import os

# Constantes
ESC_KEY = 27  # Code ASCII pour la touche ESC
SPC_KEY = 32  # Code ASCII pour la touche ESPACE

### 1.2/ Affichage du flux vidéo

Dans un premier temps, nous voulons pouvoir récupérer le flux d’une caméra vidéo (assurez-
vous d’en avoir une connectée à votre ordinateur). Pour ce faire, nous allons utiliser la classe
VideoCapture du module Highgui d’OpenCV.
Définissez la variable suivante :

```
// Declaring a VideoCapture object
// it can open a camera feed
// or a video file if there is a codec on the machine
VideoCapture cap;
```

Un objet de type VideoCapture peut ouvrir le flux d’une caméra connectée à l’ordinateur,
ou bien ouvrir un fichier vidéo. Nous allons l’utiliser dans le premier cas de figure ici, mais vous pourrez essayer par vous-même d’ouvrir un fichier vidéo.

Pour ce faire, nous allons utiliser la méthode open dont la documentation est ici : https://docs.opencv.org/4.x/d8/dfe/classcv_1_1VideoCapture.html

Cette méthode prend en paramètre soit :
- une chaine de caractères (std::string) contenant le nom du fichier vidéo que l’on
souhaite ouvrir ;
- un entier représentant l’identifiant du dispositif (caméra) que l’on souhaite ouvrir (en
schématisant : indice de la ou des caméra(s) connectée(s), commençant à 0).

Vérifiez ensuite que la capture a pu être effectuée. Pour ce faire, utilisez la méthode `cap.isOpened()`
retournant un booléen permettant de savoir si tout s’est bien passé.

Modifiez votre programme pour demander en boucle à l’utilisateur un numéro de caméra
tant que la capture n’est pas ouverte (ou -1 pour arrêter complètement l’application).
Avant toute chose, nous souhaitons manipuler des images issues de la caméra. Pour ce faire,
déclarez deux objets de type `Mat` qui permettent de stocker des images en OpenCV.

```
// The current image
// retrieved from the video capture
// or read from a file
Mat image;
// The image converted into grayscale (see if we use it)
Mat gray_image;
```


Nous allons aussi avoir besoin d’une fenêtre pour afficher les résultats, pour ce faire, nous allons utiliser la méthode namedWindow du module highgui, qui permet de créer une fenêtre,
laquelle est identifiée par une chaîne de caractères représentant son titre :

```
// Creating a window to display the images
windowName = "OpenCV Calibration";
namedWindow(windowName, CV_WINDOW_AUTOSIZE);
```

La fenêtre ne doit être créée qu’une seule fois !

Pour afficher une image dans une fenêtre préalablement créée, on utilisera la méthode imshow du module highgui. Cette méthode prend en paramètre le nom de la fenêtre dans laquelle on veut afficher, et l’image que l’on souhaite afficher. Par exemple, (on supposera que image contient bel et bien une image) :

```
// Showing the image in the window
imshow(windowName,image);
```

Enfin, pour extraire une image de la capture, il suffit d’utiliser l’opérateur `>>` de la classe `VideoCapture` :

```
// Getting the new image from the camera
cap » image;
```

Remarque : vous pouvez également utiliser la méthode read (à la place de l’opérateur ») de
la classe `VideoCapture` :

```
// Getting the new image from the camera
cap.read(image);
```

La dernière chose que vous devez mettre en uvre est d’encapsuler la récupération d’une
image, puis son affichage dans une boucle. Cette dernière doit s’arrêter lorsque l’utilisateur appuie sur la touche ’ESC’ de son clavier. Pour pouvoir récupérer les entrées clavier de l’utilisateur, nous allons utiliser une nouvelle méthode du module highgui : `waitKey`. Cette méthode prend comme argument un entier qui représente le délai pendant lequel la méthode va attendre un évènement utilisateur. Si cet entier est négatif ou nul, alors la méthode est bloquante, et `waitKey` attend indéfiniment pour l’évènement.

Enfin, afin de libérer les ressources que vous avez allouées pendant la création de cette
application, n’oubliez pas d’appeler les méthodes suivantes, pour libérer successivement les fenêtres que vous avez créées, et la capture vidéo :

```
// Destroying the windows
destroyWindow(windowName);
// Releasing the video capture
cap.release();
```

Vous devriez maintenant voir le flux vidéo de votre webcam s’afficher dans la fenêtre OpenCV créée.

Nous allons rajouter une dernière étape à notre affichage de flux vidéo : la possibilité de
transformer notre image en niveaux de gris, et de choisir si l’on souhaite afficher l’image en couleurs ou en niveaux de gris. La conversion d’une image de couleur en niveaux de gris est très simple en OpenCV, il suffit d’utiliser la méthode `cvtColor` du module `imgproc`. Cette méthode prend en entrée l’image à convertir, l’image résultante et un paramètre déterminant les différentes options de conversion (voir la documentation pour plus de détails)

Ajoutez dans votre code la ligne qui permet de convertir la matrice `image` en une matrice
`gray_image` en niveaux de gris.

Enfin, nous voulons que lorsque l’utilisateur appuie sur la touche ’g’ alors on affiche l’image en niveaux de gris. Si l’on appuie une nouvelle fois sur ’g’, alors on réaffiche l’image en couleurs. Notez que la fonction `waitKey` retourne un caractère.

**Vous rendrez le code de cette première partie et ajouterez des captures d’écrans
dans votre compte rendu pour montrer le bon fonctionnement de votre code.**

In [4]:
if __name__ == "__main__":
    # Initialiser la caméra
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Impossible d'ouvrir la caméra")
        exit()
    
    # Créer une fenêtre
    window_name = "Calibration de la camera"
    cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)
    
    # Variables pour le basculement en niveaux de gris
    is_grayscale = False

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Impossible de recevoir l'image (fin du flux?). Sortie ...")
            break

        # Convertir en niveaux de gris si le basculement est actif
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        display_frame = gray_frame if is_grayscale else frame

        # Afficher l'image
        cv2.imshow(window_name, display_frame)

        # Vérifier l'entrée utilisateur
        key = cv2.waitKey(1)
        if key == ESC_KEY:  # Sortir sur ESC
            break
        elif key == ord('g'):  # Basculer en niveaux de gris sur 'g'
            is_grayscale = not is_grayscale

    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()

Résultats avec couleurs :

![Image en couleurs](cam_avec_couleurs.png)

Résultats en nuances de gris après l'appui sur la touche 'g' :

![Image en nuances de gris](cam_avec_nuances_gris.png)

*Qu'est ce que je vais faire de tout cet argent*

### 1.3/ Affichage de photos

Dans cette deuxième partie de prise en main, nous allons modifier notre programme afin d’afficher des fichiers image à la place d’un flux vidéo. Pour ce faire, vous utiliserez les images fournies sur le serveur pédagogique dans le dossier "calib_gopro". Créez un nouveau main à partir du travail de la partie précédente.

Pour lire un fichier image, nous allons utiliser la méthode `imread` du module highgui.

Modifiez votre méthode main afin que votre boucle d’affichage du flux vidéo de la caméra,
affiche en boucle les images contenues dans le dossier "calib_gopro". Ces images sont toutes nommées GOPR84, suivi d’un numéro (précédé de 0 pour les numéros inférieurs à 10) puis possèdent l’extension .JPG. Il existe 27 images allant de GOPR8401.JPG à GOPR8427.JPG.

Pour cette partie, il se peut que vous deviez utiliser une méthode de conversion d’un nombre entier en chaine de caractères. Cela est faisable grâce à la méthode `to_string` de la bibliothèque standard de C++ (depuis C++11) :

```
std::string msg = "bli";
int monEntier = 12;
bli += std::to_string(monEntier));
```

**Vous rendrez également le code de cette deuxième partie.**

In [10]:
def display_images(image_folder):
    """
    Affiche les images d'un dossier en boucle dans une fenêtre.
    :param image_folder: Chemin vers le dossier contenant les images.
    """
    # Liste des fichiers d'images
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.JPG')])

    if not image_files:
        print("Aucune image trouvée dans le dossier spécifié.")
        return

    # Nom de la fenêtre
    window_name = "Image Viewer"
    cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)

    # Index pour parcourir les images
    index = 0
    total_images = len(image_files)

    while True:
        # Chargement de l'image courante
        image_path = os.path.join(image_folder, image_files[index])
        image = cv2.imread(image_path)

        if image is None:
            print(f"Impossible de lire l'image : {image_path}")
            index = (index + 1) % total_images
            continue

        # Affichage de l'image
        cv2.imshow(window_name, image)

        # Lecture de l'entrée utilisateur
        key = cv2.waitKey(1000)  # Attente de 1 seconde entre chaque image
        if key == ESC_KEY:  # Touche ESC pour quitter
            break
        elif key == SPC_KEY:  # Touche Espace pour passer à l'image suivante
            index = (index + 1) % total_images
        else:
            # Passer automatiquement à l'image suivante après le délai
            index = (index + 1) % total_images

    # Libération des ressources
    cv2.destroyAllWindows()


if __name__ == "__main__":
    # Chemin du dossier contenant les images
    folder_path = "calib_gopro"  # Remplacez par le chemin de votre dossier d'images
    display_images(folder_path)

Résultats :

![Affichage des images](affichage_images.png)

Dans la suite du TP, afin d’illustrer l’intérêt de la calibration de caméra, vous devrez utiliser les photos du dossier "calib_gopro", et les comparer avec les résultats que vous obtiendrez sur les autres fichiers d’exemples présents sur le serveur pédagogique.

## 2/ Calibration d'une caméra en OpenCV

Après cette étape de prise en main, nous allons réaliser la calibration de la caméra proprement dite. Nous allons :
1. détecter les coins d’un échiquier
2. une fois que assez d’images d’échiquiers seront détectées nous allons pouvoir calculer
les paramètres intrinsèques de la caméra et les coefficients de distorsion
3. redresser l’image de la caméra d’entrée

### 2.1/ Détection d'un échiquier